# RFM 分析

使用 UCI Online Retail 資料集

* [https://archive.ics.uci.edu/ml/datasets/Online+Retail](https://archive.ics.uci.edu/ml/datasets/Online+Retail)

參考資料：

* https://en.wikipedia.org/wiki/RFM_(market_research)

## 下載資料集

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

--2020-10-13 17:58:14--  https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23715344 (23M) [application/x-httpd-php]
Saving to: ‘Online Retail.xlsx’

Online Retail.xlsx  100%[===================>]  22.62M  24.5MB/s    in 0.9s    

2020-10-13 17:58:16 (24.5 MB/s) - ‘Online Retail.xlsx’ saved [23715344/23715344]



## 讀取資料集

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Retail_df = pd.read_excel("Online Retail.xlsx")

In [ ]:
Retail_df['amount'] = Retail_df['Quantity']*Retail_df['UnitPrice']

In [ ]:
Retail_df.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.3


In [ ]:
Retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
 8   amount       541909 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 37.2+ MB


In [ ]:
Retail_df['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [ ]:
Retail_df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

## 刪除缺失值

In [ ]:
df = Retail_df[Retail_df['CustomerID'].notnull()]

In [ ]:
ref_date = df['InvoiceDate'].max()

## 計算 R, F, M 值

In [ ]:
# 計算最近一次購買時間
recency = df['InvoiceDate'].groupby(df['CustomerID']).max()
# 計算購買的次數
frequency = df['InvoiceDate'].groupby(df['CustomerID']).count()
# 計算購買的金額
monetary = df['amount'].groupby(df['CustomerID']).sum()

In [ ]:
# 計算上一次購買間隔天數(幾天前)
r_interval = (ref_date - recency_value).dt.days

In [ ]:
# 計算 R, F, M 值
r_score = pd.cut(r_interval, 5, labels=[5, 4, 3, 2, 1])
f_score = pd.cut(frequency_value, 5, labels=[1, 2, 3, 4, 5])
m_score = pd.cut(monetary_value, 5, labels=[1, 2, 3, 4, 5])

In [ ]:
rfm_df = pd.concat([r_score, f_score, m_score], axis=1)
rfm_df.columns = ['r_score', 'f_score', 'm_score']

In [ ]:
rfm_df = rfm_df.astype(int)

In [ ]:
rfm_df.head()

,r_score,f_score,m_score
CustomerID,,,
12346.0,1,1,1
12347.0,5,1,1
12348.0,5,1,1
12349.0,5,1,1
12350.0,1,1,1


In [ ]:
rfm_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 4372 entries, 12346.0 to 18287.0
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   r_score  4372 non-null   int64
 1   f_score  4372 non-null   int64
 2   m_score  4372 non-null   int64
dtypes: int64(3)
memory usage: 296.6 KB


## 計算 RFM 總分

兩種方式：

* 直接評估 R, F, M 的重要性，再做加權平均成為一個數字。
* 直接串接 R, F, M 各自的值，秀出一個字串。

In [ ]:
# 方法一：加權平均
rfm_df['rfm_score'] = rfm_df['r_score'] * 0.5 + rfm_df['f_score'] * 0.3 + rfm_df['m_score'] * 0.2

In [ ]:
# 方法二：RFM组合
rfm_df_tmp = rfm_df.copy()
rfm_df_tmp['r_score'] = rfm_df_tmp['r_score'].astype('string')
rfm_df_tmp['f_score'] = rfm_df_tmp['f_score'].astype('string')
rfm_df_tmp['m_score'] = rfm_df_tmp['m_score'].astype('string')
rfm_df['rfm_comb'] = rfm_df_tmp['r_score']+rfm_df_tmp['f_score']+rfm_df_tmp['m_score']

In [ ]:
rfm_df.head()

,r_score,f_score,m_score,rfm_score,rfm_comb
CustomerID,,,,,
12346.0,1,1,1,1.0,111
12347.0,5,1,1,3.0,511
12348.0,5,1,1,3.0,511
12349.0,5,1,1,3.0,511
12350.0,1,1,1,1.0,111


In [ ]:
rfm_df.sort_values('rfm_score', ascending=False)

,r_score,f_score,m_score,rfm_score,rfm_comb
CustomerID,,,,,
14911.0,5,4,3,4.3,543
17841.0,5,5,1,4.2,551
14096.0,5,4,2,4.1,542
14646.0,5,2,5,4.1,525
18102.0,5,1,5,3.8,515
...,...,...,...,...,...
12505.0,1,1,1,1.0,111
12651.0,1,1,1,1.0,111
14090.0,1,1,1,1.0,111
